### I. Set up

In [3]:
%pip install pandas
%pip install plotly
%pip install nbformat
import sys
!{sys.executable} -m pip install nbformat --upgrade
%pip install ace_tools_open


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import libraries
# Required packages: plotly, pandas
import json
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from collections import defaultdict
import ace_tools_open as tools

import level_2_learning_comparisons

import matplotlib.pyplot as plt
import numpy as np

/Users/chelseashe/Thought_Toolkit/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Logs successfully saved to: /Users/chelseashe/Thought_Toolkit/formal1_logs.json


/Users/chelseashe/Thought_Toolkit/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0
Some weights of th

In [5]:
# Load data
# Make sure the file name is correct and in the same directory as the script
with open("level_2_actions_per_session_semantic.json") as f:
    level_2_actions_per_session_semantic = json.load(f)

### II Semantic Visualization - Correlation between semantic expansion per sentence text_insert

In [19]:
# Plotting and helper functions
def plot_semantic_expansion_sentence(writing_data, session_ids):
    fig = make_subplots(
        rows=len(session_ids),
        cols=1,
        subplot_titles=[f"Session: {session_id}" for session_id in session_ids],
        vertical_spacing=0.15,
    )
    for idx, session_id in enumerate(session_ids, start=1):
        if session_id not in writing_data:
            print(f"Warning: No data found for {session_id}")
            continue

        session_actions = writing_data[session_id]
        learning_data = []

        writing_progress = 1
        for i, action in enumerate(session_actions):
            if "writing_type" in action:
                if action["writing_type"] == "backstage":
                    if action["level_1_action_type"] == "insert_text_human_ai_prompt":
                        learning_data.append(
                            {
                                "writing_progress": writing_progress,
                                "action_type": action.get("action_type", ""),
                                "level_1_action_type": "human_insert_backstage",
                                "semantic_expansion": action["semantic_expansion"]
                            }
                        )
                        writing_progress += 1
                    elif action["level_1_action_type"] == "present_suggestion":
                        learning_data.append(
                            {
                                "writing_progress": writing_progress,
                                "action_type": action.get("action_type", ""),
                                "level_1_action_type": "ai_suggest_backstage",
                                "semantic_expansion": action["semantic_expansion"],
                            }
                        )
                        writing_progress += 1
                    # NEED TO IMPLEMENT THESE LATER
                    # elif action["level_1_action_type"] == "delete_text"
                    # elif action["level_1_action_type"] == "human_edit_ai"
                elif action["writing_type"] == "frontstage":
                    if action["level_1_action_type"] == "insert_text_human" and (not "level_2_action_type" in action or action["level_2_action_type"] != "human_edit_ai"):
                        for sentence in action["action_modified_sentences"]:
                            learning_data.append(
                                {
                                    "writing_progress": writing_progress,
                                    "action_type": action.get("action_type", ""),
                                    "level_1_action_type": "human_insert_frontstage",
                                    "semantic_expansion": action[
                                        "semantic_expansion_info"
                                    ][sentence],
                                }
                            )
                            writing_progress += 1
                            added = False
                    elif "level_2_action_type" in action and action["level_2_action_type"] == "move_frontstage":
                        learning_data.append(
                            {
                                "writing_progress": writing_progress,
                                "action_type": action.get("action_type", ""),
                                "level_1_action_type": "ai_insert_moved_to_frontstage",
                                "semantic_expansion": action["semantic_expansion"],
                            }
                        )
                        writing_progress += 1
                    elif action["level_1_action_type"] == "delete_text":
                        learning_data.append(
                            {
                                "writing_progress": writing_progress,
                                "action_type": action.get("action_type", ""),
                                "level_1_action_type": "human_delete_frontstage",
                                "semantic_expansion": action["semantic_expansion"],
                            }
                        )
                        writing_progress += 1
                    elif "level_2_action_type" in action and action["level_2_action_type"] == "human_edit_ai":
                        learning_data.append(
                            {
                                "writing_progress": writing_progress,
                                "action_type": action.get("action_type", ""),
                                "level_1_action_type": "human_insert_edit_frontstage",
                                "semantic_expansion": action["semantic_expansion"],
                            }
                        )
                        writing_progress += 1

        learning_data = sorted(learning_data, key=lambda x: x["writing_progress"])
        # print(learning_data)

        if not learning_data:
            print(f"Warning: No data found for {session_id}")
            continue

        data_df = pd.DataFrame(learning_data)

        # Graph individual cognitive measures
        y_max = 0.0
        aggregate_score = 0.0
        backstage_addition = 0.0
        aggregate_data = []
        for learning_index, action in enumerate(learning_data):
            details = {
                "writing_progress": action["writing_progress"],
                "level_1_action_type": action["level_1_action_type"],
                "aggregate_semantic_expansion_frontend": aggregate_score,
                "increment_semantic_expansion": action["semantic_expansion"],
            }
            if "backstage" in action["level_1_action_type"]:
                if learning_index>0 and learning_data[learning_index-1]:
                    details["aggregate_semantic_expansion_frontend"] += backstage_addition
                backstage_addition += action["semantic_expansion"]
            if "frontstage" in action["level_1_action_type"] and learning_index > 0 and learning_data[learning_index - 1]:
                backstage_addition = 0.0
                aggregate_score += action["semantic_expansion"]

            aggregate_data.append(details)
            if aggregate_score > y_max:
                y_max = aggregate_score
        aggregate_df = pd.DataFrame(aggregate_data)
        print(f"{session_id}")

        colors = {
            "human_insert_backstage": "#64B5F6",
            "ai_suggest_backstage": "#ff8fab",
            "human_insert_frontstage": "#023e8a",
            "ai_insert_moved_to_frontstage": "#c1121f",
            "human_delete_frontstage": "#9d4edd",
            "human_insert_edit_frontstage": "#5a189a",
        }

        for action_type in colors.keys():
            actions = [
                action
                for action in aggregate_data
                if action.get("level_1_action_type") == action_type
            ]
            for index in range(len(actions)-1,-1,-1):
                x0 = actions[index]["writing_progress"] - 1
                x1 = actions[index]["writing_progress"]
                y0 = actions[index]["aggregate_semantic_expansion_frontend"]
                y1 = y0 + actions[index]["increment_semantic_expansion"]

                fig.add_trace(
                    go.Scatter(
                        x=[x0, x1],
                        y=[y0, y1],
                        mode="lines+markers",                # ← line with a dot
                        line=dict(color=colors[action_type], width=3),
                        marker=dict(size=7, color=colors[action_type]),
                        showlegend=False,                    # keep your existing legend
                        hovertemplate="x=%{x}<br>y=%{y}<extra></extra>",
                    ),
                    row=idx,
                    col=1,
                )

        if idx == 1:
            for color, hex in colors.items():
                fig.add_trace(
                    go.Scatter(
                        x=[None],
                        y=[None],
                        mode="lines",
                        line=dict(color=hex, width=2),
                        name=color,
                        showlegend=True,
                    ),
                    row=1,
                    col=1,
                )

    fig.update_layout(
        plot_bgcolor="white",
        legend_title_font_color="rgba(0, 0, 0, 0)",
        height=500 * len(session_ids),
        width=1800,
        showlegend=True,
        title={
            "text": "Semantic Expansion over Writing Sessions<br>"
            + '<span style="font-size: 12px;">*Measuring how well humans are learning while writing with GenAI</span>',
        },
    )

    fig.update_xaxes(
        title=None,
        range=[-1, data_df["writing_progress"].max()+1],
        mirror=False,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="white",
    )

    fig.update_xaxes(
        title="Progress into the writing session", row=len(session_ids), col=1
    )

    fig.update_yaxes(
        title="Scores",
        mirror=False,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="white",
    )

    fig.show()
    return fig

In [20]:
writing_data = level_2_actions_per_session_semantic
# Session IDs: An array of sessions you wish to plot.
# Adjust as needed. Check session IDs in data exploration for options.
session_ids = [
    "legislation_corporate_1",
    "legislation_corporate_2",
    "legislation_corporate_3",
    "legislation_corporate_4",
    "legislation_corporate_5",
    "legislation_corporate_6",
    "legislation_antitrust_1",
    "legislation_antitrust_2",
    "legislation_antitrust_3",
    "legislation_antitrust_4",
    "legislation_antitrust_5",
    "legislation_antitrust_6",
    "legislation_antitrust_7",
    "legislation_antitrust_8",
]

# Generate the plot
for session_id in session_ids:
    fig = plot_semantic_expansion_sentence(writing_data, [session_id])

legislation_corporate_1


legislation_corporate_2


legislation_corporate_3


legislation_corporate_4


legislation_corporate_5


...

legislation_antitrust_4


legislation_antitrust_5


legislation_antitrust_6


legislation_antitrust_7


legislation_antitrust_8
